In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import QuantileTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
SEED = 30

In [ ]:
FILE_DIR = '../MoA/lish-moa/'

In [ ]:
from kerastuner import HyperModel


class MyHyperModel(HyperModel):

    def __init__(self, inputshape,outputshape,init):
        self.inputshape = inputshape
        self.outputshape = outputshape
        self.init = init

    def build(self, hp):
        
        model = keras.models.Sequential()
        model.add(keras.layers.Input(shape=(self.inputshape,)))
        for i in range(3):
        
            model.add(keras.layers.BatchNormalization())
        
            model.add(keras.layers.Dense(units= hp.Int('units',min_value = 512,max_value=2048,step=256),
                                         activation="relu",
                                         kernel_initializer="he_normal"))
        
            model.add(keras.layers.Dropout(rate = hp.Choice('rate',values=[0.3,0.4,0.5])))
            

        model.add(keras.layers.Dense(self.outputshape,activation='sigmoid',bias_initializer=self.init))
        
        model.compile(
            optimizer=keras.optimizers.Adam(
                hp.Choice('learning_rate',
                          values=[3e-2, 3e-3, 3e-4])),
            loss=loss = keras.losses.BinaryCrossentropy(label_smoothing=0.001),
            metrics=logloss)
        return model



In [ ]:
hypermodel = MyHyperModel(num_classes=10)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    directory='my_dir',
    project_name='helloworld')

tuner.search(x, y,
             epochs=5,
             validation_data=(val_x, val_y))